In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import time

In [2]:
# device setting
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # cuda는 gpu
device

device(type='cuda')

In [3]:
# Hyper-parameter
num_epochs = 5 # 에폭을 다섯번 돌리겠다
batch_size = 4 # 에폭의 단위
learning_rate = 0.001

In [4]:
# data preprocesssing
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean=(0.5, 0.5, 0.5), std =(0.5, 0.5, 0.5))]) # transforms = tensor로 바꾸어줌 mean과 std가 세가지 값이 들어간 이유는 채널수가 3이여서 여기선 (rgb)


In [5]:
# # Cifar 10 : 60000 32 X 32 color images for 10 classes # 이미지일때 (3 , x, x) -> 여기서는 (3,32,32) torchvison => 이미지 처리
# train_dataset = torchvision.datasets.CIFAR10(root ='./',
#                                              train = True,
#                                              download = True,
#                                              transform = transform)

# test_dataset = torchvision.datasets.CIFAR10(root ='./',
#                                              test = False,
#                                              download = True,
#                                              transform = transform)
train_dataset = torchvision.datasets.CIFAR10(root='./data',
                                             train=True,
                                             download=True,
                                             transform=transform)

test_dataset = torchvision.datasets.CIFAR10(root='./data',
                                             train=False,
                                             download=True,
                                             transform=transform)



100%|██████████| 170498071/170498071 [00:02<00:00, 80143855.92it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [6]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True) # dataloader 배치단위로 데이터를 학습 시키거나 데이터를 공급해줌 훈련용데이터는 규칙적인 패턴이 있을수도 있기에 shuffle = True
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# test_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
# #batch단위로 데이터를 제공

In [7]:
# 모델정의
class CNN(nn.Module):
  def __init__(self): # 데이터의 실행 재료들
    super(CNN, self).__init__()
    #self를 붙이면 전역변수로 사용 가능
    self.conv1 = nn.Conv2d(in_channels= 3, out_channels= 6, kernel_size= 5) # out channel 은 필터의 수 하이퍼파라미터 사용자가 지정하는 수 파라미터 학습되어지는 수 # kernel_size = 5x5
    self.pool = nn.MaxPool2d(2, 2) # (krnel size, 스트라이드값)
    self.conv2 = nn.Conv2d(6, 16, 5) #x conv1채널의 수(out_channel = 6)가 6이기 떄문에 in_channel = 6이다
    self.fc1 = nn.Linear(16*5*5, 100) # 16차원 5x5의 flatten, 100
    self.fc2 = nn.Linear(100,50)
    self.fc3 = nn.Linear(50,10) # 마지막은 클래스의 숫자

  # 실행시킬 구조 (순전파)
  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x))) # conv1 -> ReLu실행 -> maxpooling 실행-> conv2 -> 반복 -> flatten -> f1 -> f2 -> f3 - > 예측
    x = self.pool(F.relu(self.conv2(x))) # ReLu는 shape이 바뀌지 않는다.
    #1차원 백터로 flatten
    x.view(-1,16*5*5) # (배치사이즈 = -1 ,fc1) -> 1차원 백터로 전환 -1은 배치사이즈 -> (배치사이즈, flatten의 값)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

# class CNN(nn.Module):
#   def __init__(self):
#     super(CNN, self).__init__()
#     self.conv1 = nn.Conv2d(in_channels= 3, out_channels= 6, kernel_size= 5) #3:RGB, 6:필터 수(지정), 5:5x5의 필터 사용
#     self.pool = nn.MaxPool2d(2, 2) #(사이즈(2x2), 스트라이드 (2칸씩))
#     self.conv2 = nn.Conv2d(6, 16, 5) #6:위애서 필터 수가 6이므로 인채널도6, 16:필터 수
#     self.fc1 = nn.Linear(16*5*5, 100)
#     self.fc2 = nn.Linear(100, 50)
#     self.fc3 = nn.Linear(50, 10) #Cifar10의 클래스가 10이므로 10

#   def forward(self, x):
#     x = self.pool(F.relu(self.conv1(x)))
#     x = self.pool(F.relu(self.conv2(x)))
#     x = x.view(-1, 16*5*5) #Flatten #-1:batch
#     x = F.relu(self.fc1(x))
#     x = F.relu(self.fc2(x))
#     x = self.fc3(x)
#     return x


In [8]:
# 높이 공식과 너비의 공식) (W(열의 수 즉 높이)-F(필터의 사이즈) + 2*P(패딩의 개수))/S(스트레이트값)) + 1 for Conv
# ((W-(필터의 size))/S)+1 for pooling -> 예 Maxpool2d(size,스트라이드값)

In [9]:
test_dataset[0][0].shape

torch.Size([3, 32, 32])

In [10]:
model = CNN().to(device) # device에 올린다
model

CNN(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=10, bias=True)
)

In [11]:
# 손실함수
critertion = nn.CrossEntropyLoss() # CNN은 분류이기 때문에 CrossEntropyLoss()사용 회귀면 MSE, MAE
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate) # adam = > w(가중치)의 최저값을 구해줌 (경사하강법)

In [16]:
#  train
n_total_steps = len(train_loader)
for epoch in range(num_epochs): # 에폭의 수의 반복
  for i, (images, labels) in enumerate(train_loader): # 반복을 할때마다 train_loader를 반복함 - 반복은 배치사이즈로 데이터를 반복함 (image, labels) 은 (x축, y축)
    images = images.to(device) # divice로 데이터로 보냄
    labels = labels.to(device)

    #forward(순전파) 단계
    outputs = model(images)
    loss = criterion(outputs, labels) # 예측값과 실제값을 예측결과를 확인함

    #역전파파 단계
    optimizer.zero_grad() # 기울기 초기화
    loss.backward() # 오차를 전 단계로 이동시킴 -> 역전파
    optimizer.step() # 업데이트시킴

    if (i + 1) % 3000 == 0:
      print(f'Epoch: {epoch + 1}/ {num_epochs}, Loss: {loss.item(): 3f}')
print("Finished Training")

#train
# n_total_steps = len(train_loader)
# for epoch in range(num_epochs): #에폭을 다음 만큼 반복
#   for i, (images, labels) in enumerate(train_loader): #배치사이즈로 공급
#     images = images.to(device)
#     labels = labels.to(device)

#     outputs = model(images)
#     loss = criterion(outputs, labels)

#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()
#     if(i+1) % 3000 == 0:
#       print(f'Epoch: {epoch+1}/{num_epochs}, Step: {i+1}/{n_total_steps}, Loss: {loss.item(): 3f}')

# print("Finished Training!")

RuntimeError: ignored

In [ ]:
# 평가단계
with torch.no_grad(): # 평가할때는 업데이트가 필요 없기에 기울기를 계산하지 않는다 -> grad는 기울기 no_grad
  n_correct = 0
  n_samples = 0

  model.eval() # 학습을 평가할때 사용
  # 실행 안될때 각 줄에 print 써보기
  for images, labels in test_loader:
    images = images.to(device)
    images = images.to(device)

    outputs = model(images)

    _, pred = torch.max(outputs, 1) # max(outputs , 0과 1) -> row와 culumn 즉 dimension -> 토치 맥스는 인덱스 값도 줌,  _, == not의 의미이고 변수로 쓰지 않겠다. 즉 인덱스 번화와 값중 값(pred만 가져온다)
    n_correct += (pred == labels).sum().item() # .itemd = 값만 가져옴
    n_samples += labels.size(0)

  acc = n_correct / n_samples * 100
  print(f'ACC: {acc:.3f}')